In [6]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
warnings.filterwarnings('ignore')
%precision 3

'%.3f'

In [7]:
## data前処理
path = "../data/"
df = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')

Mr = df[df["Name"].str.contains("Mr.")]["Age"].mean()
Mrs = df[df["Name"].str.contains("Mrs.")]["Age"].mean()
Mas = df[df["Name"].str.contains("Master.")]["Age"].mean()
Miss = df[df["Name"].str.contains("Miss.")]["Age"].mean()
Dr = df_test[df_test["Sex"]=="male"]["Age"].mean()

Mr = df_test[df_test["Name"].str.contains("Mr.")]["Age"].mean()
Mrs = df_test[df_test["Name"].str.contains("Mrs.")]["Age"].mean()
Mas = df_test[df_test["Name"].str.contains("Master.")]["Age"].mean()
Miss = df_test[df_test["Name"].str.contains("Miss")]["Age"].mean()

df.loc[df["Age"].isnull() & df["Name"].str.contains("Mr"), "Age"] = Mr
df.loc[df["Age"].isnull() & df["Name"].str.contains("Mrs"), "Age"] = Mrs
df.loc[df["Age"].isnull() & df["Name"].str.contains("Master"), "Age"] = Mas
df.loc[df["Age"].isnull() & df["Name"].str.contains("Miss"), "Age"] = Miss
df.loc[df["Age"].isnull() & df["Name"].str.contains("Dr."), "Age"] = Dr

print(f"Mrs is {Mrs} Miss is {Miss} Mas {Mas} Mrs {Mrs}")
df_test.loc[df_test["Age"].isnull() & df_test["Name"].str.contains("Mr"), "Age"] = Mr
df_test.loc[df_test["Age"].isnull() & df_test["Name"].str.contains("Mrs"), "Age"] = Mrs
df_test.loc[df_test["Age"].isnull() & df_test["Name"].str.contains("Master"), "Age"] = Mas
df_test.loc[df_test["Age"].isnull() & df_test["Name"].str.contains("Miss"), "Age"] = Miss
df_test.loc[df_test["Age"].isnull() & df_test["Name"].str.contains("Ms."), "Age"] = (Mrs+Miss)/2


df["Embarked"] = df["Embarked"].fillna("S")
df_test["Embarked"] = df_test["Embarked"].fillna("S")

df["family"] = df["SibSp"] + df["Parch"]
df_test["family"] = df_test["SibSp"] + df_test["Parch"]

df = pd.get_dummies(df,columns=["Embarked"],dtype=int)
df_test = pd.get_dummies(df_test,columns=["Embarked"],dtype=int)

df.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)
df_test.replace({'Sex': {'male': 0, 'female': 1}}, inplace=True)
df_test['Fare'].fillna(27.079812, inplace=True)

dropCols = ["Name","Ticket","Cabin","SibSp","Parch"] 
 
df_sub = df.drop(columns=dropCols)
df_test_sub = df_test.drop(columns=dropCols)

X = df_sub.iloc[:, 2:].values
y = df_sub["Perished"].values
df_test_X = df_test_sub.iloc[:, 1:].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)

Mrs is 38.903225806451616 Miss is 21.774843750000002 Mas 7.406470588235294 Mrs 38.903225806451616


In [ ]:
###kfold実装
kf = KFold(n_splits=5,shuffle=True,random_state=42)

score = []
models = []

param_grid = {'max_depth': [3, 5, 7,9],
              'min_samples_leaf': [1, 2, 4]}

for fold_ , (train_index,valid_index) in enumerate(kf.split(X,y)):
    print(f'fold{fold_+1}start')
    train_x = X[train_index]
    valid_x = X[valid_index]
    train_y = y[train_index]
    valid_y = y[valid_index]
    
    train_data = lgb.Dataset(train_x,train_y)
    eval_data = lgb.Dataset(valid_x,valid_y)
    
    params = {
    'objective': 'binary',
    'boosting_type':'gbdt',
    'num_leaves':15,
    'max_depth':5,
    'random_state':0,
    'min_data_in_leaf':20,
    'n_estimators': 50,
    
}
    
    model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=[train_data,eval_data]
)
    
    print(f"training ------ \n{accuracy_score(train_y,(model.predict(train_x)>0.5).astype(int))}")
    
    pred = model.predict(valid_x)
    result = (pred>0.5).astype(int)
    
    print(f"test -------\n{accuracy_score(valid_y,result)}")
    score.append(round(accuracy_score(valid_y,result),2))
    models.append(model)

In [ ]:
#ハイパーパラメータチューニング
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import lightgbm as lgb

# ハイパーパラメータの設定
param_grid = {
    'max_depth': [3, 5],
    'min_data_in_leaf': [15, 20, 35,40],
    'n_estimators': [40,50,60],
    'num_leaves': [5,10,15, 20]  # max_depth=5の場合、2^5-1=31 以下
}

# LightGBMモデルのインスタンス作成
estimator = lgb.LGBMClassifier(objective='binary')

# GridSearchCVの設定
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=5, scoring='accuracy')

# グリッドサーチの実行
grid_search.fit(X_train, y_train)

# 最適なパラメータの表示
print(f"Best parameters found: {grid_search.best_params_}")

# 最適なモデルの取得
best_model = grid_search.best_estimator_

# テストデータで予測
y_pred = best_model.predict(X_valid)

# モデルの精度を評価
accuracy = accuracy_score(y_valid, y_pred)
print(f"Test Accuracy: {accuracy}")



In [91]:
y_pred = (model.predict(df_test_X)>0.5).astype(int)
path = '../'
submission = pd.read_csv(path + 'gender_submission.csv')
submission['Perished'] = y_pred
submission.to_csv('./submission.csv', index=False)